In [22]:
import pandas as pd
import datetime as dt
import sqlite3
import datetime as dt
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

In [23]:
kpp = pd.read_csv(r'C:\Users\sgasmi\Desktop\Extractions\KPP.csv', sep="|")
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str})
mapping_id = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Mapping+des+ID+.xlsx", skiprows=4)

In [24]:
fafr = fafr[fafr["Code Contrat"].isin(["STA2", "STA1"])==False].reset_index(drop=True)  # Exclusion des stagiaires

## Préparation de la base effectif

In [25]:
res = fafr[["Matricule", "Nom", "Prénom", "Etablissement"]]

In [26]:
res = res.drop_duplicates()

## Préparation du fichier KPP

In [27]:
kpp.loc[kpp["enddate"].isna()==True, "enddate"] = dt.datetime(2050, 1, 1) # On remplace les dates de fin des lignes actives

In [28]:
kpp = kpp.merge(mapping_id[["Username", "Payroll ID"]], left_on="username", right_on="Username", how="left")

In [29]:
kpp["Today"] = dt.date.today()

In [30]:
kpp = kpp.drop("Username", axis=1)

In [31]:
col = [x.replace(" ", "_") for x in kpp.columns]
kpp.colums = col

<ipython-input-31-59a1a84ff462>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  kpp.colums = col


In [32]:
#Création d'une base de donnée in memory
conn = sqlite3.connect(':memory:')
# Ajout des Dataframe dans la base de données
kpp.to_sql('kpp', conn, index=False)
# Création d'une requête sql
req = "select * from kpp where Today between startdate and enddate"

C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [33]:
# Exécution de la requête et stokage du résultat dans un DataFrame
kpp = pd.read_sql_query(req, conn)
kpp = kpp.drop("Today", axis=1)

In [34]:
kpp= kpp.loc[kpp["Payroll ID"].isna()==False].copy()

In [35]:
kpp= kpp.reset_index(drop=True)

In [36]:
ref_code = kpp["referential"].unique()

In [37]:
kpp["Payroll ID"] = kpp["Payroll ID"].apply(lambda x: x[4:])
fafr["Matricule"] = fafr["Matricule"].astype("str")

In [38]:
#kpp.to_excel(r"C:\Users\sgasmi\Desktop\Export Python\KPP.xlsx")
#fafr.to_excel(r"C:\Users\sgasmi\Desktop\Export Python\fafr.xlsx")

In [39]:
kpp2 = kpp.loc[kpp["referential"]=="Grade"]
res = fafr.merge(kpp2[["Payroll ID", "username","referential"]], left_on=["Matricule"], right_on=["Payroll ID"], how="left")

In [40]:
result = pd.DataFrame()
fafr["Matricule"] = fafr["Matricule"].astype("str")
for i, code in enumerate(ref_code):
    kpp2 = kpp.loc[kpp["referential"]==code]
    res = fafr.merge(kpp2[["Payroll ID", "username","referential"]], left_on=["Matricule"], right_on=["Payroll ID"], how="left")
    res = res.reset_index(drop=True)
    res = res.loc[res["referential"].isna()==True]
    res["Anomalie"] = str(code) + " vide"
    res = res[["Matricule", "Nom", "Prénom", "BU", "Etablissement", "Nom de l'ent. juridique", "Anomalie"]]
    if i==0:
        result = res
    else :
        result = result.append(res)
    

In [48]:
today = dt.date.today()
today = today.strftime('%d %m %Y')
today = today.replace(" ", "")

In [49]:
resize_column([result], "Anomalies MyHR", r"C:\Users\sgasmi\Desktop\Power BI applications\Sources MyHR\Anomalie_MyHR_" + str(today) + ".xlsx")

kpp